# Predictions

## Data wrangling

In [2]:
# Preamble
import pandas as pd
import numpy as np
pd.set_option("mode.chained_assignment", None)
import random
random.seed(1509)
import matplotlib.pyplot as plt
import lightgbm as lgb
import pyarrow.feather as feather
from os import chdir, getcwd
import statsmodels.api as sm
from pprint import pprint
from nested_cv import NestedCV

# sci-kit
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn import tree

In [3]:
data_dir = '/home/jovyan/work/Data/'
results_dir = '/home/jovyan/work/Results/'

In [4]:
# select_features = ['reporter.ISO', 'partner.ISO', 'year',
#                    'ln.Tot_IFF_t', 'ln.In_Tot_IFF_t',
#                    'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
#                    'dist', 'contig', 
#                    'comlang', 'comcol', 'col45', 
#                    'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
#                    'rCorrCont', 'pCorrCont',
#                    'rRegQual', 'pRegQual', 
#                    'rRuleLaw', 'pRuleLaw',
#                    'rSecrecyScore', 'pSecrecyScore',
#                    'rFSI.rank', 'pFSI.rank',
#                    'rKFSI13', 'pKFSI13',
#                    'rKFSI17', 'pKFSI17',
#                    'rKFSI20', 'pKFSI20',
#                    'rFATF', 'pFATF',
#                    'ihs.tariff',
#                    'kai_o', 'kai_d', 'kao_o', 'kao_d',
#                    'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
#                    'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']
select_features = ['reporter.ISO', 'partner.ISO', 'year',
                   'ln.Tot_IFF_t', 'ln.In_Tot_IFF_t',
                   'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
                   'dist', 'contig', 
                   'comlang', 'comcol', 'col45', 
                   'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
                   'rCorrCont', 'pCorrCont',
                   'rRegQual', 'pRegQual', 
                   'rRuleLaw', 'pRuleLaw',
                   'pSecrecyScore',
                   'pFSI.rank',
                   'pKFSI13',
                   'pKFSI17',
                   'pKFSI20',
                   'rFATF', 'pFATF',
                   'ihs.tariff',
                   'kai_o', 'kai_d', 'kao_o', 'kao_d',
                   'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
                   'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']

features = [       'ln.gdp_o', 'ln.gdp_d', 'ln.pop_o', 'ln.pop_d', 
                   'dist', 'contig', 
                   'comlang', 'comcol', 'col45', 
                   'ihs.entry_cost_o', 'ihs.entry_cost_d', 'rta',
                   'rCorrCont', 'pCorrCont',
                   'rRegQual', 'pRegQual', 
                   'rRuleLaw', 'pRuleLaw',
                   'pSecrecyScore',
                   'pFSI.rank',
                   'pKFSI13',
                   'pKFSI17',
                   'pKFSI20',
                   'rFATF', 'pFATF',
                   'ihs.tariff',
                   'kai_o', 'kai_d', 'kao_o', 'kao_d',
                   'cc_o', 'cc_d', 'cci_o', 'cci_d', 'cco_o', 'cco_d',
                   'di_o', 'di_d', 'dii_o', 'dii_d', 'dio_o', 'dio_d']

ids = [       'reporter.ISO', 'partner.ISO', 'year']

## Subset sample

In [5]:
data = feather.read_feather(results_dir + 'Africa_agg.feather')

In [6]:
def create_smp(data, features):
    """
    Create train and test samples that are complete.
    """
    smp = data[features]
    smp.dropna(axis=0, how='any', inplace=True)
    return smp

In [7]:
data_smp = create_smp(data, select_features)

In [8]:
idx = data_smp[ids]
X = data_smp[features]
Y_out = data_smp[['ln.Tot_IFF_t']]
Y_in = data_smp[['ln.In_Tot_IFF_t']]

In [51]:
feather.write_feather(idx, results_dir + 'idx.feather')
feather.write_feather(X, results_dir + 'X.feather')
feather.write_feather(Y_out, results_dir + 'Y_out.feather')
feather.write_feather(Y_in, results_dir + 'Y_in.feather')

In [50]:
print('X: ', X.shape)
print('Y_out: ', Y_out.shape)
print('idx: ', idx.shape)

X:  (5333, 42)
Y_out:  (5333, 1)
idx:  (4256, 3)


In [381]:
Y_out.shape

(5333, 1)

In [382]:
Y_in.shape

(5333, 1)

### Train/test split

In [11]:
train_agg = feather.read_feather(results_dir + 'train_agg.feather')
test_agg = feather.read_feather(results_dir + 'test_agg.feather')

In [12]:
train_agg_smp = create_smp(train_agg, select_features)
test_agg_smp = create_smp(test_agg, select_features)

In [13]:
feather.write_feather(train_agg_smp, results_dir + 'train_agg_smp.feather')
feather.write_feather(test_agg_smp, results_dir + 'test_agg_smp.feather')

In [14]:
Y_train_out = train_agg_smp[['ln.Tot_IFF_t']]
Y_train_in = train_agg_smp[['ln.In_Tot_IFF_t']]
X_train = train_agg_smp[features]
Y_test_out = test_agg_smp[['ln.Tot_IFF_t']]
Y_test_in = test_agg_smp[['ln.In_Tot_IFF_t']]
X_test = test_agg_smp[features]

In [48]:
feather.write_feather(X_train, results_dir + 'X_train.feather')
feather.write_feather(Y_train_out, results_dir + 'Y_train_out.feather')

In [15]:
print('X_train: ', X_train.shape, '\nX_test: ',  X_test.shape)

X_train:  (4256, 42) 
X_test:  (1077, 42)


In [49]:
idx = train_agg_smp[ids]

## Linear regression

### Fit linear regression model

In [39]:
linear_mod_out = LinearRegression()  
linear_mod_out.fit(X.values, Y_out.values)

LinearRegression()

In [ ]:
linear_mod_in = LinearRegression()  
linear_mod_in.fit(X_train.values, Y_train_in.values)

### Print coefficients

In [ ]:
print(linear_mod_out.intercept_)
print(linear_mod_out.coef_)

In [ ]:
print(linear_mod_in.intercept_)
print(linear_mod_in.coef_)

In [ ]:
# Xconst = sm.add_constant(X_train)
# est = sm.OLS(Y_train_out, Xconst)
# est2 = est.fit()
# print(est2.summary())

In [ ]:
# Xconst = sm.add_constant(X_train)
# est = sm.OLS(Y_train_in, Xconst)
# est2 = est.fit()
# print(est2.summary())

### Predictions

In [ ]:
preds_LM_train_out = linear_mod_out.predict(X_train)
preds_LM_test_out = linear_mod_out.predict(X_test)

In [ ]:
preds_LM_train_in = linear_mod_in.predict(X_train)
preds_LM_test_in = linear_mod_in.predict(X_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_LM_train_out), results_dir + 'preds.LM.train_out_agg.feather')
feather.write_feather(pd.DataFrame(preds_LM_test_out), results_dir + 'preds.LM.test_out_agg.feather')

In [ ]:
feather.write_feather(pd.DataFrame(preds_LM_train_in), results_dir + 'preds.LM.train_in_agg.feather')
feather.write_feather(pd.DataFrame(preds_LM_test_in), results_dir + 'preds.LM.test_in_agg.feather')

### Predictive accuracy

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_LM_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_LM_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_LM_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_LM_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_LM_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_LM_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_LM_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_LM_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_LM_test_out))  
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_LM_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_LM_test_out))
print('Dollar RMSE of the training set (outflows):', np.mean(np.square((np.exp(Y_train_out.values)*10**3 - np.exp(preds_LM_train_out)*10**3))) / 10**9)
print('Dollar RMSE of the test set (outflows):', np.mean(np.square((np.exp(Y_test_out.values)*10**3 - np.exp(preds_LM_test_out)*10**3))) / 10**6)

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_LM_test_in))
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_LM_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_LM_test_in))
print('Dollar RMSE of the test set (inflows):', np.mean(np.square((np.exp(Y_train_in.values)*10**3 - np.exp(preds_LM_train_in)*10**3))) / 10**9)
print('Dollar RMSE of the test set (inflows):', np.mean(np.square((np.exp(Y_test_in.values)*10**3 - np.exp(preds_LM_test_in)*10**3))) / 10**9)

### Cross-validation

In [ ]:
linear_mod_out = LinearRegression()  
linear_mod_out.fit(X.values, Y_out.values)

In [45]:
scores = cross_val_score(linear_mod_out, X, Y_out.values.ravel(), cv = 10)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.45186556  0.4127257   0.52329192  0.55024428  0.51445511  0.51008895
  0.41924502  0.50052528  0.48595955 -0.0046091 ]


In [46]:
scores.mean()

0.43637922845408933

In [ ]:
predictions = cross_val_predict(linear_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [ ]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.LM.CV_out.feather')

## Random Forests

### Fit baseline random forests regression

In [10]:
RF_0_mod_out = RandomForestRegressor(random_state = 1509)
RF_0_mod_out.fit(X, Y_out.values.ravel())

In [11]:
RF_0_mod_in = RandomForestRegressor(random_state = 1509)
RF_0_mod_in.fit(X, Y_in.values.ravel())

### Tune hyperparameters with randomized search

In [17]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 3000, num = 100)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 500, num = 100)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 10)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(1, 100, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 40, 70, 100, 130, 161, 191, 221, 251, 281, 312, 342, 372, 402, 432, 463, 493, 523, 553, 583, 614, 644, 674, 704, 734, 765, 795, 825, 855, 885, 916, 946, 976, 1006, 1036, 1067, 1097, 1127, 1157, 1187, 1218, 1248, 1278, 1308, 1338, 1369, 1399, 1429, 1459, 1489, 1520, 1550, 1580, 1610, 1640, 1671, 1701, 1731, 1761, 1791, 1822, 1852, 1882, 1912, 1942, 1973, 2003, 2033, 2063, 2093, 2124, 2154, 2184, 2214, 2244, 2275, 2305, 2335, 2365, 2395, 2426, 2456, 2486, 2516, 2546, 2577, 2607, 2637, 2667, 2697, 2728, 2758, 2788, 2818, 2848, 2879, 2909, 2939, 2969, 3000], 'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425

In [20]:
# Create the base model to tune
RF_0_mod_out = RandomForestRegressor(random_state = 1509)

# Random search of parameters on base model using 3 fold cross validation 
# Search across 100 different combinations, and use all available cores
RF_random_out = RandomizedSearchCV(random_state = 1509,
                                   estimator = RF_0_mod_out, 
                                   param_distributions = random_grid,
                                   scoring = 'r2',
                                   n_iter = 100,
                                   verbose = 3, n_jobs = -1)

In [ ]:
# Fit the random search model
RF_random_out.fit(X, Y_out.values.ravel())

In [21]:
# Fit the random search model
RF_random_out.fit(X_train, Y_train_out.values.ravel())

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END bootstrap=True, max_depth=475, max_features=sqrt, min_samples_leaf=45, min_samples_split=12, n_estimators=372;, score=0.530 total time=   2.1s
[CV 3/5] END bootstrap=True, max_depth=475, max_features=sqrt, min_samples_leaf=45, min_samples_split=12, n_estimators=372;, score=0.515 total time=   2.0s
[CV 5/5] END bootstrap=True, max_depth=475, max_features=sqrt, min_samples_leaf=45, min_samples_split=12, n_estimators=372;, score=0.523 total time=   2.0s
[CV 2/5] END bootstrap=True, max_depth=195, max_features=auto, min_samples_leaf=1, min_samples_split=12, n_estimators=1278;, score=0.678 total time=  56.0s
[CV 4/5] END bootstrap=True, max_depth=195, max_features=auto, min_samples_leaf=1, min_samples_split=12, n_estimators=1278;, score=0.689 total time=  57.5s
[CV 1/5] END bootstrap=False, max_depth=430, max_features=auto, min_samples_leaf=78, min_samples_split=2, n_estimators=2093;, score=0.460 total time= 1.2min


RandomizedSearchCV(estimator=RandomForestRegressor(random_state=1509),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15, 20, 25, 30, 35,
                                                      40, 45, 50, 55, 60, 65,
                                                      70, 75, 80, 85, 90, 95,
                                                      100, 105, 110, 115, 120,
                                                      125, 130, 135, 140, 145,
                                                      150, ...],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 12, 23, 34, 45,
                                                             56, 67, 78, 89,
                                                             100],
                                        'min_samples_split': 

[CV 5/5] END bootstrap=False, max_depth=500, max_features=auto, min_samples_leaf=89, min_samples_split=50, n_estimators=2788;, score=0.497 total time= 1.4min
[CV 3/5] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=100, min_samples_split=39, n_estimators=2214;, score=0.495 total time=  40.8s
[CV 5/5] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=100, min_samples_split=39, n_estimators=2214;, score=0.512 total time=  40.3s
[CV 2/5] END bootstrap=True, max_depth=460, max_features=auto, min_samples_leaf=89, min_samples_split=2, n_estimators=1218;, score=0.508 total time=  22.7s
[CV 4/5] END bootstrap=True, max_depth=460, max_features=auto, min_samples_leaf=89, min_samples_split=2, n_estimators=1218;, score=0.551 total time=  23.4s
[CV 1/5] END bootstrap=True, max_depth=470, max_features=auto, min_samples_leaf=89, min_samples_split=23, n_estimators=644;, score=0.488 total time=  12.1s
[CV 3/5] END bootstrap=True, max_depth=470, max_features=a

In [115]:
# Mean cross-validated score of the best_estimator (5-fold cross-validation by default)
print('Best Score: %s' % RF_random_out.best_score_)

Best Score: 0.6762124915340071


In [369]:
# The hyperparameters were found using a cross-validation randomized search strategy on the training sample X_train
RF_out_tuned = RandomForestRegressor(random_state = 1509,
                                     n_estimators = 1278,
                                     max_depth = 195,
                                     min_samples_split = 12,
                                     min_samples_leaf = 1,
                                     max_features = 'auto',
                                     bootstrap = True)

In [370]:
# Fit tuned model on training data
RF_out_tuned.fit(X_train, Y_train_out.values.ravel())

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [371]:
best_random = RF_out_tuned

In [116]:
# Save best estimator that gave highest score on left out data
best_random = RF_random_out.best_estimator_

In [372]:
best_random

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [373]:
# Check and evaluate score using cross-validation (5-fold cross-validation by default)
scores = cross_val_score(RF_random_out.best_estimator_, X_train, Y_train_out.values.ravel())

In [374]:
scores.mean()

0.6762124915340071

In [375]:
# Check and evaluate score using cross-validation (5-fold cross-validation by default) on full sample
scores = cross_val_score(RF_random_out.best_estimator_, X, Y_out.values.ravel())

In [376]:
scores.mean()

0.2998644516891945

In [366]:
RF_random_out.best_params_

{'n_estimators': 1278,
 'min_samples_split': 12,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 195,
 'bootstrap': True}

In [ ]:
RF_random_out.cv_results_

In [377]:
print(best_random.score(X_train, Y_train_out))

0.8918552097958046


In [380]:
# Refit tuned and traning model on training data
RF_out_tuned.fit(X_train, Y_train_out.values.ravel())

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [378]:
print(best_random.score(X_test, Y_test_out))

0.7081047138987878


In [379]:
print(best_random.score(X, Y_out))

0.8539825727395456


In [32]:
cross_val_score(RF_random_out.best_estimator_, X_train, Y_train_out.values.ravel(), cv = 3)

array([0.64285616, 0.67522511, 0.68364949])

In [33]:
cross_val_score(RF_random_out.best_estimator_, X_train, Y_train_out.values.ravel(), cv = 5)

array([0.64624182, 0.67827449, 0.67677421, 0.6886621 , 0.69110983])

In [383]:
predictions_CV = cross_val_predict(best_random, X_train, Y_train_out.values.ravel())
r2_score(Y_train_out, preds.predictions_CV)

0.6770733080450901

In [392]:
(mean_squared_error(Y_train_out, predictions_CV))

3.232962735490173

In [389]:
preds_test = RF_out_tuned.predict(X_test)

In [393]:
(mean_squared_error(Y_test_out, preds_test))

2.998418505852522

In [44]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.RF.CV_out.feather')

# Placebo

In [347]:
Xi_placebo = X_train[['ln.gdp_o', 'ln.pop_o', 'ihs.entry_cost_o', 
              'rCorrCont', 'rRegQual', 'rRuleLaw',
              'rFATF', 
              'kai_o', 'kao_o', 'cc_o', 'cci_o', 'cco_o', 'di_o', 'dii_o', 'dio_o']]

In [348]:
Xj_placebo = X_train[['ln.gdp_d', 'ln.pop_d', 'ihs.entry_cost_d', 
                      'pCorrCont', 'pRegQual', 'pRuleLaw',
                      'pSecrecyScore', 'pFSI.rank', 'pKFSI13', 'pKFSI17', 'pKFSI20',
                      'pFATF', 
                      'kai_d', 'kao_d', 'cc_d', 'cci_d', 'cco_d', 'di_d', 'dii_d', 'dio_d']]

In [349]:
Xij_placebo = X_train[['dist', 'contig', 'comlang', 'comcol', 'col45', 'rta',
               'ihs.tariff']]

In [350]:
Xi_placebo = Xi_placebo.sample(frac = 1)
Xj_placebo = Xj_placebo.sample(frac = 1)
Xij_placebo = Xij_placebo.sample(frac = 1)
X_placebo = pd.concat([Xi_placebo, Xj_placebo, Xij_placebo], axis = 1, ignore_index = True)

In [334]:
Y_placebo = Y_train_out.sample(frac = 1)

In [ ]:
Y_test_placebo = Y_t_out.sample(frac = 1)

In [227]:
values = X_train.values

In [230]:
X_placebo = np.apply_along_axis(np.random.permutation, 0, values)

In [233]:
X_placebo = pd.DataFrame(X_placebo)

In [355]:
X_placebo = feather.read_feather(results_dir + 'X_placebo.feather')

In [358]:
X_placebo

,ln.gdp_o,ln.gdp_d,ln.pop_o,ln.pop_d,dist,contig,comlang,comcol,col45,ihs.entry_cost_o,...,cci_o,cci_d,cco_o,cco_d,di_o,di_d,dii_o,dii_d,dio_o,dio_d
0,17.002892,20.776837,9.812471,9.964075,15950.811523,0,0,0,0,5.599176,...,0.0,0.0,1.0,1.0,0.5,0.5,0.0,1.0,1.0,0.0
1,18.940779,21.218547,10.745254,14.068912,12967.769531,0,0,0,0,2.936674,...,0.0,1.0,1.0,1.0,0.5,1.0,0.0,1.0,1.0,1.0
2,16.477952,22.194992,10.595192,11.757877,11404.291016,0,0,0,0,5.283229,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,16.477952,21.566940,10.595192,11.060651,7168.276855,0,0,0,0,5.283229,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,19.472103,20.005454,10.797742,9.121301,10380.564453,0,0,0,0,2.658165,...,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,16.883266,21.352833,10.534197,14.040364,5691.471191,0,1,1,0,5.053736,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4252,19.279345,21.354591,11.282159,14.015309,4432.625000,0,0,0,0,3.037256,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4253,16.365225,19.638761,7.139608,10.316525,5478.541016,0,0,1,0,1.529660,...,0.0,1.0,0.0,1.0,0.5,1.0,1.0,1.0,0.0,1.0
4254,14.967163,22.232182,8.697428,14.096663,11656.857422,0,0,0,0,6.219799,...,0.0,1.0,1.0,1.0,0.5,1.0,0.0,1.0,1.0,1.0


In [335]:
# best_random is an estimator with tuned hyperparameters
best_random

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [360]:
best_random.fit(X_placebo, Y_train_out.values.ravel())

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [361]:
# training error
best_random.score(X_placebo, Y_train_out)

0.6325592235438458

In [363]:
# test error
best_random.score(X_test, Y_test_out)

0.009554751408330175

In [364]:
scores = cross_val_score(best_random, X, Y_out.values.ravel())

In [365]:
scores.mean()

0.2998644516891945

In [222]:
best_random.score(X_test, Y_test_out)

-0.28633500762419306

In [136]:
r2 = []
for i in range(100):
    X_placebo = X_test.sample(frac = 1)
    score = best_random.score(X_placebo, Y_test_out)
    r2.append(score)

In [225]:
best_random.fit(shuffled_train_df, Y_train_out.values.ravel())

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [226]:
best_random.score(shuffled_train_df, Y_train_out)

0.6339378235034976

### Tune hyperparameters with grid search

In [47]:
# Create the base model to tune (with some parameters from random search)
RF_base_mod_out  = RandomForestRegressor(random_state = 1509,
                                         max_features = 'auto',
                                         bootstrap = True)

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [500, 2500],
    'max_depth': [10, 300],
    'min_samples_split': [8, 80],
    'min_samples_leaf': [8, 80],
}

# Grid search of parameters on base model using 3 fold cross validation 
# Search all possible cominations, and use all available cores
RF_grid_out = GridSearchCV(estimator = RF_base_mod_out, 
                           param_grid = param_grid,
                           scoring = 'r2',
                           verbose = 3, n_jobs = -1)

In [48]:
# Fit the grid search model
RF_grid_out.fit(X, Y_out.values.ravel())

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=RandomForestRegressor(random_state=1509), n_jobs=-1,
             param_grid={'max_depth': [10, 300], 'min_samples_leaf': [8, 80],
                         'min_samples_split': [8, 80],
                         'n_estimators': [500, 2500]},
             scoring='r2', verbose=3)

[CV 2/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=500;, score=0.296 total time=  40.2s
[CV 4/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=500;, score=0.381 total time=  39.9s
[CV 1/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.481 total time= 3.3min
[CV 3/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.552 total time= 3.3min
[CV 5/5] END max_depth=10, min_samples_leaf=8, min_samples_split=8, n_estimators=2500;, score=0.448 total time= 3.3min
[CV 1/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.490 total time= 2.8min
[CV 3/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.543 total time= 2.8min
[CV 5/5] END max_depth=10, min_samples_leaf=8, min_samples_split=80, n_estimators=2500;, score=0.417 total time= 2.8min
[CV 2/5] END max_depth=10, min_samples_leaf=80,

In [49]:
print('Best Score: %s' % RF_grid_out.best_score_)

Best Score: 0.4384566873340452


In [50]:
RF_grid_out.best_params_

{'max_depth': 300,
 'min_samples_leaf': 8,
 'min_samples_split': 80,
 'n_estimators': 2500}

In [65]:
best_grid = RF_grid_out.best_estimator_
print(best_grid.score(X, Y_out))

0.6113986943864123


In [66]:
best_grid.fit(X, Y_out.values.ravel())

RandomForestRegressor(max_depth=300, min_samples_leaf=8, min_samples_split=80,
                      n_estimators=2500, random_state=1509)

In [67]:
preds_RF_out = best_grid.predict(X)
r2_score(Y_out, preds_RF_out)

0.7057793564482469

In [52]:
scores = cross_val_score(best_grid, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [0.37440636 0.3426772  0.48558579]


In [53]:
scores.mean()

0.4008897832777349

### Manually configure hyperparameters

In [45]:
RF_mod_out = RandomForestRegressor(random_state = 1509,
                                   n_estimators = 1278,
                                   max_depth = 195,
                                   min_samples_split = 12,
                                   min_samples_leaf = 1,
                                   max_features = 'auto',
                                   bootstrap = True)

In [46]:
# Fit the manually configured model
RF_mod_out.fit(X, Y_out.values.ravel())

RandomForestRegressor(max_depth=195, min_samples_split=12, n_estimators=1278,
                      random_state=1509)

In [80]:
print(RF_mod_out.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 125, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 23, 'min_samples_split': 23, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1278, 'n_jobs': None, 'oob_score': False, 'random_state': 1509, 'verbose': 0, 'warm_start': False}


In [47]:
preds_RF_out = RF_mod_out.predict(X)
r2_score(Y_out, preds_RF_out)

0.8962271032343396

In [ ]:
predictions = cross_val_predict(RF_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [64]:
scores = cross_val_score(RF_mod_out, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.17849742 -0.12171697  0.22869577]


### Predictions

In [ ]:
# Base model
# preds_RF_train_out = RF_0.predict(X_train)
# preds_RF_test_out = RF_0.predict(X_test)

In [ ]:
# Manually configured model (outflows)
preds_RF_train_out = RF_mod_out.predict(X_train)
preds_RF_test_out = RF_mod_out.predict(X_test)

In [ ]:
# Model tuned with randomized and grid search (outflows)
preds_RF_train_out = RF_grid_out.predict(X_train)
preds_RF_test_out = RF_grid_out.predict(X_test)

In [ ]:
# Manually configured model (inflows)
preds_RF_train_in = RF_mod_in.predict(X_train)
preds_RF_test_in = RF_mod_in.predict(X_test)

In [ ]:
feather.write_feather(pd.DataFrame(preds_RF_train_out), results_dir + 'preds.RF.train_out_agg.feather')
feather.write_feather(pd.DataFrame(preds_RF_test_out), results_dir + 'preds.RF.test_out_agg.feather')

In [ ]:
feather.write_feather(pd.DataFrame(preds_RF_train_in), results_dir + 'preds.RF.train_in_agg.feather')
feather.write_feather(pd.DataFrame(preds_RF_test_in), results_dir + 'preds.RF.test_in_agg.feather')

### Predictive accuracy

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_RF_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_RF_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_RF_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_RF_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_RF_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_RF_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_RF_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_RF_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_RF_test_out))
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_RF_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_RF_test_out))
# print('Dollar RMSE of the training set (outflows):', np.mean(np.square((np.exp(Y_train_out.values) - np.exp(preds_RF_train_out)))))
# print('Dollar RMSE of the test set (outflows):', np.mean(np.square((np.exp(Y_test_out.values) - np.exp(preds_RF_test_out)))))

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_RF_test_in))  
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_RF_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_RF_test_in))
#print('Dollar RMSE of the training set (inflows):', np.mean(np.square(np.exp(Y_train_in.values)/10**3 - np.exp(preds_RF_train_in)/10**3)))
#print('Dollar RMSE of the test set (inflows):', np.mean(np.square(np.exp(Y_test_in.values)/10**3 - np.exp(preds_RF_test_in)/10**3)))

### Cross-validation

In [ ]:
RF_mod_CV = RandomForestRegressor(random_state = 1509,
                                  n_estimators = 1278,
                                  max_depth = 125,
                                  min_samples_split = 23,
                                  min_samples_leaf = 23,
                                  max_features = 'auto',
                                  bootstrap = True)

In [ ]:
scores = cross_val_score(RF_mod_CV, X, Y_out.values.ravel(), cv = 6)
print('Cross-validated scores:', scores)

In [ ]:
scores.mean()

In [68]:
predictions = cross_val_predict(best_grid, X, Y_out.values.ravel(), cv = 3)
r2_score(Y_out, predictions)

0.21085539345854087

In [69]:
feather.write_feather(pd.DataFrame(predictions), results_dir + 'preds.RF.CV_out.feather')

### Visualize results

In [ ]:
feat_importances = pd.Series(best_random.feature_importances_, index = X.columns)
indices = np.argsort(-1*feat_importances)
names = [X_train.columns[i] for i in indices]
fig, ax = plt.subplots(figsize = (10, 10))
plt.barh(range(X_train.shape[1]), feat_importances[indices], color = 'red')
plt.yticks(range(X_train.shape[1]), names, fontsize = 12)
plt.gca().invert_yaxis()
ax.set_xlabel('Feature importance')
plt.title("Random Forest - Feature importance for illicit outflows", fontsize = 16)
plt.savefig(results_dir + 'RF_feature_importance_out.png')

In [ ]:
feat_importances = pd.Series(RF_mod_in.feature_importances_, index=X_train.columns)
indices = np.argsort(-1*feat_importances)
names = [X_train.columns[i] for i in indices]
fig, ax = plt.subplots(figsize = (10, 10))
plt.barh(range(X_train.shape[1]), feat_importances[indices], color = 'red')
plt.yticks(range(X_train.shape[1]), names, fontsize = 12)
plt.gca().invert_yaxis()
ax.set_xlabel('Feature importance')
plt.title("Random Forest - Feature importance for illicit inflows", fontsize = 16)
plt.savefig(results_dir + 'RF_feature_importance_in.png')

In [ ]:
tree.export_graphviz(RF_mod_out.estimators_[0])

In [ ]:
tree.plot_tree(RF_mod_out.estimators_[0])

### Nested cross-validation

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 3000, num = 100)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 500, num = 100)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [int(x) for x in np.linspace(2, 50, num = 10)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(1, 100, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the parameter grid
param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'max_features': max_features,
              'bootstrap': bootstrap}

In [ ]:
# Perform nested cross-validation
RF_mod_NCV = NestedCV(model = RandomForestRegressor(random_state = 1509), 
                      params_grid = param_grid,
                      outer_kfolds = 5, inner_kfolds = 5,
                      n_jobs = -1,
                      cv_options={'metric': r2_score,
                                  'randomized_search': True,
                                  'randomized_search_iter': 10})

In [ ]:
# Fit the nested cross-validated model
RF_mod_NCV.fit(X = X, y = Y_out.values.ravel())

In [ ]:
RF_mod_NCV.outer_scores

In [ ]:
np.mean(RF_mod_NCV.outer_scores)

In [ ]:
RF_mod_NCV.best_params

## Light GBM

In [ ]:
lightGBM_train_out = lgb.Dataset(X_train, Y_train_out)
lightGBM_test_out = lgb.Dataset(X_test, Y_test_out)
lightGBM_train_in = lgb.Dataset(X_train, Y_train_in)
lightGBM_test_in = lgb.Dataset(X_test, Y_test_in)

In [ ]:
params = {
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 100,
    'max_depth': 10,
    'learning_rate' : 0.1,
    'feature_fraction' : 0.6,
    'verbosity' : -1,
    'random_state' : 1509
}
lightGBM_mod_out = lgb.train(
    params,
    lightGBM_train_out,
    500,
    valid_sets = [lightGBM_train_out, lightGBM_test_out],
    valid_names = ["train", "test"],
    early_stopping_rounds = 50,
    verbose_eval = 500
)

In [ ]:
params = {
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 100,
    'max_depth': 10,
    'learning_rate' : 0.1,
    'feature_fraction' : 0.6,
    'verbosity' : -1,
    'random_state' : 1509
}
lightGBM_mod_in = lgb.train(
    params,
    lightGBM_train_in,
    500,
    valid_sets = [lightGBM_train_in, lightGBM_test_in],
    valid_names = ["train", "test"],
    early_stopping_rounds = 50,
    verbose_eval = 500
)

In [63]:
preds_lightGBM_out = lightGBM_mod_out.predict(X)
r2_score(Y_out, preds_lightGBM_out)

0.722053692562466

In [ ]:
predictions = cross_val_predict(lightGBM_mod_out, X, Y_out.values.ravel(), cv = 6)
r2_score(Y_out, predictions)

In [64]:
scores = cross_val_score(lightGBM_mod_out, X, Y_out.values.ravel(), cv = 3)
print('Cross-validated scores:', scores)

Cross-validated scores: [ 0.17849742 -0.12171697  0.22869577]


In [ ]:
preds_lightGBM_train_out = pd.DataFrame(lightGBM_mod_out.predict(X_train))
preds_lightGBM_test_out = pd.DataFrame(lightGBM_mod_out.predict(X_test))

In [ ]:
preds_lightGBM_train_in = pd.DataFrame(lightGBM_mod_in.predict(X_train))
preds_lightGBM_test_in = pd.DataFrame(lightGBM_mod_in.predict(X_test))

In [ ]:
feather.write_feather(preds_lightGBM_train_out, results_dir + 'preds.lightGBM.train_out_agg.feather')
feather.write_feather(preds_lightGBM_test_out, results_dir + 'preds.lightGBM.test_out_agg.feather')

In [ ]:
feather.write_feather(preds_lightGBM_train_in, results_dir + 'preds.lightGBM.train_in_agg.feather')
feather.write_feather(preds_lightGBM_test_in, results_dir + 'preds.lightGBM.test_in_agg.feather')

In [ ]:
print("RMSE of the training set (outflows):", np.sqrt(mean_squared_error(Y_train_out, preds_lightGBM_train_out)))
print("R^2 of the training set (outflows):", r2_score(Y_train_out, preds_lightGBM_train_out))
print("RMSE of the test set (outflows):", np.sqrt(mean_squared_error(Y_test_out, preds_lightGBM_test_out)))
print("R^2 of the test set (outflows):", r2_score(Y_test_out, preds_lightGBM_test_out))

In [ ]:
print("RMSE of the training set (inflows):", np.sqrt(mean_squared_error(Y_train_in, preds_lightGBM_train_in)))
print("R^2 of the training set (inflows):", r2_score(Y_train_in, preds_lightGBM_train_in))
print("RMSE of the test set (inflows):", np.sqrt(mean_squared_error(Y_test_in, preds_lightGBM_test_in)))
print("R^2 of the test set (inflows):", r2_score(Y_test_in, preds_lightGBM_test_in))

In [ ]:
print('Mean Absolute Error (outflows):', metrics.mean_absolute_error(Y_test_out, preds_lightGBM_test_out))
print('Mean Squared Error of the training set (outflows):', mean_squared_error(Y_train_out, preds_lightGBM_train_out))
print('Mean Squared Error of the test set (outflows):', mean_squared_error(Y_test_out, preds_lightGBM_test_out))
#print('Dollar RMSE of the training set (outflows):', np.mean(np.square(np.exp(Y_train_out.values)*10**3 - np.exp(preds_lightGBM_train_out)*10**3)) / 10**6)
#print('Dollar RMSE of the test set (outflows):', np.mean(np.square(np.exp(Y_test_out.values)*10**3 - np.exp(preds_lightGBM_test_out)*10**3)) / 10**6)

In [ ]:
print('Mean Absolute Error (inflows):', metrics.mean_absolute_error(Y_test_in, preds_lightGBM_test_in))
print('Mean Squared Error of the training set (inflows):', mean_squared_error(Y_train_in, preds_lightGBM_train_in))
print('Mean Squared Error of the test set (inflows):', mean_squared_error(Y_test_in, preds_lightGBM_test_in))
#print('Dollar RMSE of the training set (inflows):', np.mean(np.square(np.exp(Y_train_in.values)/10**3 - np.exp(preds_lightGBM_train_in)/10**3)))
#print('Dollar RMSE of the test set (inflows):', np.mean(np.square(np.exp(Y_test_in.values)/10**3 - np.exp(preds_lightGBM_test_in)/10**3)))

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
lgb.plot_importance(lightGBM_mod_out, height = 0.8, ax = ax)
ax.grid(False)
plt.title("LightGBM - Feature importance for illicit outflows", fontsize = 16)
plt.savefig(results_dir + 'LightGBM_feature_importance_out.png')

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
lgb.plot_importance(lightGBM_mod_in, height = 0.8, ax = ax)
ax.grid(False)
plt.title("LightGBM - Feature importance for illicit inflows", fontsize = 16)
plt.savefig(results_dir + 'LightGBM_feature_importance_in.png')